In [2]:
# !pip install datasets
# !pip install transformers
# !pip install snac
# !pip install soundfile
# !pip install torchaudio --index-url https://download.pytorch.org/whl/cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 12.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 26.6 MB/s  0:00:017.0 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.5 MB/s  0:00:0035.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [datasets];237m━━ 16/17 [datasets]ce-hub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 23.4 MB/s  0:00:00.6 MB/s eta 0:00:0102
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 49.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [transformers]0m 3/4 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [snac]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s  0:00:00m 4.8 MB/s eta 0:00:01


In [1]:
from datasets import load_dataset, Audio
from transformers import MimiModel, AutoFeatureExtractor

import torch
from snac import SNAC
import torchaudio
import soundfile as sf

In [2]:
filepath = "00001_001_sales_000_1.wav"
target_sample_rate = 24000

# Load the audio file
waveform, sample_rate = torchaudio.load(filepath)

# Resample if necessary
if sample_rate != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    waveform = resampler(waveform)

# Ensure it's mono
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

audio = waveform.squeeze(0)

In [3]:
audio.shape

torch.Size([334383])

In [4]:
model = MimiModel.from_pretrained("kyutai/mimi")
feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")

model.safetensors:   0%|          | 0.00/385M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [5]:
feature_extractor.sampling_rate

24000

In [6]:
# pre-process the inputs
inputs = feature_extractor(raw_audio=audio, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt")

In [7]:
audio

tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0017, -0.0021, -0.0006])

In [8]:
inputs["input_values"][0][0]

tensor([ 0.0000,  0.0000,  0.0000,  ..., -0.0017, -0.0021, -0.0006])

In [29]:
# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"])
audio_values = model.decode(encoder_outputs.audio_codes)[0]

In [31]:
output_filepath = "mimi.wav"
# --- 4. Save the Reconstructed Audio ---
print(f"Saving reconstructed audio to: {output_filepath}")
audio_to_save = audio_values.squeeze().cpu().detach().numpy()
sf.write(output_filepath, audio_to_save, target_sample_rate)
print("Done.")

Saving reconstructed audio to: mimi.wav
Done.
